In [1]:
from surprise import Dataset, Reader
import pandas as pd
from collections import defaultdict

from UserDefinedNMF import UserDefinedNMF

In [2]:
def load_data(file_path):
    df = pd.read_csv(file_path, usecols=['userId', 'movieId', 'rating'])
    return df

In [3]:
def prepare_data_for_surprise(dataframe):
    reader = Reader(rating_scale=(1, 5))
    data = Dataset.load_from_df(dataframe[['userId', 'movieId', 'rating']], reader)
    return data

In [4]:
traindf = load_data('./datasets/training_data.csv')

data = prepare_data_for_surprise(traindf)

In [5]:
user_ids = traindf['userId'].unique()
user_id_map = {old: new for new, old in enumerate(user_ids)}
item_ids = traindf['movieId'].unique()
item_id_map = {old: new for new, old in enumerate(item_ids)}

traindf['userId'] = traindf['userId'].map(user_id_map)
traindf['movieId'] = traindf['movieId'].map(item_id_map)

In [6]:
model = UserDefinedNMF()

model.fit(data.build_full_trainset())

In [7]:
u, i = 1, 2
prediction = model.predict(u, i)
print(f"Predicted rating for user {u} and item {i} is {prediction}")

Predicted rating for user 1 and item 2 is 4.232164319295247


In [8]:
testdf = load_data('./datasets/testing_data.csv')
testdf['userId'] = testdf['userId'].map(user_id_map, na_action='ignore')
testdf['movieId'] = testdf['movieId'].map(item_id_map, na_action='ignore')
trainset = data.build_full_trainset()
testset = [tuple(row) for row in testdf.itertuples(index=False)]

In [9]:
predictions = model.test(testset)

In [10]:
predictions_df = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est'])
predictions_df.describe()

,uid,iid,rui,est
count,18043.000000,18043.000000,18043.000000,18043.000000
mean,321.772377,1099.526354,3.563847,3.743933
std,182.610173,878.233695,1.030476,0.500235
min,0.000000,0.000000,0.500000,1.437549
25%,169.000000,360.000000,3.000000,3.425468
50%,317.000000,868.000000,4.000000,3.783507
75%,476.000000,1696.000000,4.000000,4.096492
max,609.000000,3298.000000,5.000000,5.000000


In [11]:
def calculate_accuracy(predictions, threshold=0.5):
    correct_predictions = sum(abs(estimated_rating - true_rating) <= threshold for _, _, true_rating, estimated_rating in predictions)
    accuracy = correct_predictions / len(predictions)
    return accuracy

In [12]:
threshold = 0.5
test_accuracy = calculate_accuracy(predictions, threshold)

print(f"Accuracy (within ±{threshold} of actual rating): {test_accuracy:.2%}")

Accuracy (within ±0.5 of actual rating): 48.16%


In [13]:
def create_predictions_dataframe(predictions):
    predictions_data = [{'uid': pred[0], 'iid': pred[1], 'og_rating': pred[2], 'nmf_rating': pred[3]} for pred in predictions]
    return pd.DataFrame(predictions_data)

test_pred_df = create_predictions_dataframe(predictions)

In [14]:
df_movies = pd.read_csv('./datasets/Movies.csv')

In [15]:
def get_top_n(predictions, n=10):
    top_n = defaultdict(list)
    
    for row in predictions.itertuples(index=False):
        uid, iid, nmf_rating = row.uid, row.iid, row.nmf_rating
        top_n[uid].append((iid, nmf_rating))
    
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)  
        top_n[uid] = user_ratings[:n]
    
    return top_n

In [16]:
def show_user_recommended_movies_in_df(predictions, user_id, n=10):
    top_n = get_top_n(predictions, n)
    
    if user_id in top_n:
        recommended_movies_ids = [movie_id for movie_id, _ in top_n[user_id]]
    else:
        return pd.DataFrame()
    
    recommended_movies_df = df_movies.loc[df_movies['movieId'].isin(recommended_movies_ids)]
    
    return recommended_movies_df

In [17]:
for i in range(1,2):
    print("\t\t\t\t*****{}*****\n".format(i))
    print(show_user_recommended_movies_in_df(test_pred_df, i, 5))
    print("\n")

				*****1*****

     movieId                                              title  \
143      171                                     Jeffrey (1995)   
168      199  Umbrellas of Cherbourg, The (Parapluies de Che...   
580      711                                     Flipper (1996)   

                              genres  
143              ['Comedy', 'Drama']  
168  ['Drama', 'Musical', 'Romance']  
580        ['Adventure', 'Children']  




In [18]:
show_user_recommended_movies_in_df(test_pred_df, 403, 5)

,movieId,title,genres
24,25,Leaving Las Vegas (1995),"['Drama', 'Romance']"
299,341,Double Happiness (1994),['Drama']
331,373,Red Rock West (1992),['Thriller']
389,448,Fearless (1993),['Drama']


In [19]:
n_factors = 15
n_epochs = 30


In [33]:
from surprise import NMF, Dataset, Reader, accuracy
from surprise.model_selection import train_test_split

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(traindf[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.25)

nmf_surprise = NMF(n_factors=n_factors, n_epochs=n_epochs, verbose=True)
nmf_surprise.fit(trainset)

predictions_surprise = nmf_surprise.test(testset);
rmse_surprise = accuracy.rmse(predictions_surprise, verbose=True)


Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
RMSE: 0.9032


In [21]:
from sklearn.decomposition import NMF
from sklearn.metrics import mean_squared_error
import numpy as np

ratings_matrix_preprocessed = traindf.groupby(['userId', 'movieId'])['rating'].mean().reset_index()

ratings_matrix = ratings_matrix_preprocessed.pivot(index='userId', columns='movieId', values='rating').fillna(0)
R = ratings_matrix.values

nmf_sklearn = NMF(n_components=n_factors, max_iter=n_epochs, init='random', random_state=0)
W = nmf_sklearn.fit_transform(R)
H = nmf_sklearn.components_

predicted_ratings = np.dot(W, H)
actual_ratings = R
mse = mean_squared_error(actual_ratings, predicted_ratings)
rmse_sklearn = np.sqrt(mse)

print(f"Scikit-learn NMF RMSE: {rmse_sklearn}")



Scikit-learn NMF RMSE: 0.5577407680537037


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\decomposition\_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 30 reached. Increase it to improve convergence.
  warnings.warn(


In [22]:
def calculate_accuracy(predictions, threshold=0.5):
    correct_predictions = sum(abs(pred.est - pred.r_ui) <= threshold for pred in predictions)
    accuracy = correct_predictions / len(predictions)
    return accuracy

threshold = 0.5
test_accuracy = calculate_accuracy(predictions_surprise, threshold)
print(f"Accuracy (within ±{threshold} of actual rating): {test_accuracy:.2%}")


Accuracy (within ±0.5 of actual rating): 46.06%


In [23]:
def calculate_threshold_accuracy(actual_ratings, predicted_ratings, threshold=0.5):
    differences = np.abs(actual_ratings - predicted_ratings)
    correct_predictions = np.sum(differences <= threshold)
    total_predictions = np.product(differences.shape)
    accuracy = correct_predictions / total_predictions
    return accuracy

threshold_accuracy = calculate_threshold_accuracy(actual_ratings, predicted_ratings, threshold=0.5)
print(f"Accuracy (within ±{threshold} of actual rating): {threshold_accuracy:.2%}")


Accuracy (within ±0.5 of actual rating): 91.94%


C:\Users\ŞEVKET\AppData\Local\Temp\ipykernel_11472\741448109.py:8: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  threshold_accuracy = calculate_threshold_accuracy(actual_ratings, predicted_ratings, threshold=0.5)


In [44]:
def show_user_recommendations_sklearn(user_id, predicted_ratings, ratings_matrix, df_movies, n_recommendations=5):
    user_predicted_ratings = predicted_ratings[user_id]

    top_movie_indices = np.argsort(-user_predicted_ratings)[:n_recommendations]

    top_movie_ids = ratings_matrix.columns[top_movie_indices].tolist()

    recommended_movies_df = df_movies[df_movies['movieId'].isin(top_movie_ids)]

    return recommended_movies_df

user_id = 200
n_recommendations = 5
recommended_movies_df = show_user_recommendations_sklearn(user_id, predicted_ratings, ratings_matrix, df_movies, n_recommendations)

recommended_movies_df

,movieId,title,genres
11,12,Dracula: Dead and Loving It (1995),"['Comedy', 'Horror']"
19,20,Money Train (1995),"['Action', 'Comedy', 'Crime', 'Drama', 'Thrill..."
50,55,Georgia (1995),['Drama']
51,57,Home for the Holidays (1995),['Drama']
314,356,Forrest Gump (1994),"['Comedy', 'Drama', 'Romance', 'War']"


In [28]:
import numpy as np

def calculate_rmse(predictions):
    mse = np.mean([(true_r - est_r) ** 2 for _, _, true_r, est_r in predictions])
    return np.sqrt(mse)

In [29]:
predictions = model.test(testset)

rmse = calculate_rmse(predictions)
print(f"Model RMSE: {rmse}")


Model RMSE: 0.857694742106774
